## Question 5
You are a life actuary of an insurance company. Today on 13 March 2022, a potential
client, Alice, is looking to sign a permanent life insurance contract with your company.
Alice, born on 28 October 1983, is a smoker with the health condition standard for her
cohort with the same age, gender, and smoker status. You decide to apply the 2015
Valuation Basic Table (VBT) (www.soa.org/files/research/exp-study/2015-vbt-smokerdistinct-alb-anb.xlsx) for Alice’s mortality evaluation with the Age Nearest Birthday
(ANB) method for her age calculation. Your company applies the valuation interest rate
of 3.5% for premium pricing:

a) Calculate Alice’s annal unconditional survival rates $_np_x$ from now to age 42. [3
marks]

b) Calculate Alice’s life expectancy in years (show one decimal place). [2 marks]

c) Calculate the probability that Alice dies exactly between 6 and 9 years from now
(show five decimal places). [2 marks]

d) Calculate the minimum annual premium rate (premium as a fraction of death
benefit) that your company should charge for Alice’s cohort (show five decimal
places). [3 marks]

e) What technologies can you apply to enable a closer prediction on the company’s
cash flow linked to this line of business of permanent life insurance? [answers
should be no longer than 200 words] [4 marks]

https://en.wikipedia.org/wiki/Force_of_mortality

In [25]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from scipy.stats import norm

In [2]:
excel_df = pd.read_excel('./data/2015-vbt-smoker-distinct-alb-anb.xlsx', header=2, sheet_name='2015 FSM ANB')
excel_df.set_index = excel_df['Iss. Age']
excel_df.head()

,Iss. Age,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,Ult.,Att. Age
0,18,0.27,0.27,0.27,0.27,0.27,0.29,0.33,0.34,0.34,...,0.73,0.86,1.03,1.17,1.29,1.40,1.49,1.58,1.67,43
1,19,0.27,0.27,0.26,0.26,0.27,0.31,0.33,0.33,0.33,...,0.86,1.01,1.16,1.29,1.40,1.49,1.58,1.67,1.76,44
2,20,0.24,0.23,0.24,0.25,0.29,0.31,0.32,0.32,0.34,...,1.00,1.14,1.27,1.40,1.49,1.58,1.67,1.76,1.87,45
3,21,0.19,0.21,0.23,0.26,0.29,0.30,0.31,0.33,0.37,...,1.13,1.26,1.38,1.49,1.58,1.67,1.76,1.87,2.05,46
4,22,0.17,0.20,0.24,0.27,0.28,0.30,0.32,0.35,0.39,...,1.24,1.36,1.48,1.58,1.67,1.76,1.87,2.05,2.25,47


In [3]:
alice_bday=date(1983, 10, 28)
today = date(2022, 3, 12)
alice_age = (today- alice_bday)/timedelta(days=365)
alice_age_anb = round(alice_age)
print(f"Alice is {alice_age_anb} years old (ANB)")

Alice is 38 years old (ANB)


${_n|}q_x$: conditional mortality rate, i.e. the person is dead at the end of $(n+1)^{\text{th}}$ year given the person is alive at the end of the $n^{\text{th}}$ year

$1 - {_n|}q_x$: conditional survival rate

${_n}p_x$ the probability that a person aged $x$ is alive at the end of the $n^{\text{th}}$ year
\begin{equation}
{_n}p_x = \prod^{n-1}_{m=0}1-{_m|}q_x \quad n \geq 1
\end{equation}

$P_n$ = the probability that the person dies between year $n$ and $n+1$

In [11]:
series = excel_df.loc[excel_df['Iss. Age'] == alice_age_anb].drop(columns=['Iss. Age', 'Ult.', 'Att. Age'])
n = len(series.columns)
df = pd.DataFrame(columns=series.columns)
df.loc['${_n|}q_x$'] = series.to_numpy()[0] * 10**-2
df.loc['$1-{_n|}q_x$'] = 1 - df.loc['${_n|}q_x$'] 
df.loc['${_n}p_x$'] = df.loc['$1-{_n|}q_x$']

for i in range(1,n+1):
    df.loc['${_n}p_x$', i] = np.prod(df.loc['$1-{_n|}q_x$', list(np.arange(1, i+1))])
df.loc['$P_n$'] = df.loc['${_n}p_x$'] 
for i in range(1, n+1):
    if i == 1:
        df.loc['$P_n$', i] = 1 - df.loc['${_n}p_x$', i] 
        continue
    df.loc['$P_n$', i] = (1 - df.loc['${_n}p_x$', i]) - (1 - df.loc['${_n}p_x$', i-1])
df

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
${_n|}q_x$,0.0037,0.00530,0.007200,0.008800,0.010100,0.011300,0.012600,0.014000,0.017800,0.020400,...,0.039100,0.043600,0.048300,0.053500,0.059300,0.065900,0.073100,0.081200,0.090100,0.100100
$1-{_n|}q_x$,0.9963,0.99470,0.992800,0.991200,0.989900,0.988700,0.987400,0.986000,0.982200,0.979600,...,0.960900,0.956400,0.951700,0.946500,0.940700,0.934100,0.926900,0.918800,0.909900,0.899900
${_n}p_x$,0.9963,0.99102,0.983884,0.975226,0.965376,0.954468,0.942441,0.929247,0.912706,0.894087,...,0.742682,0.710301,0.675993,0.639828,0.601886,0.562222,0.521123,0.478808,0.435667,0.392057
$P_n$,0.0037,0.00528,0.007135,0.008658,0.009850,0.010909,0.012026,0.013194,0.016541,0.018619,...,0.030220,0.032381,0.034308,0.036166,0.037942,0.039664,0.041098,0.042315,0.043141,0.043610


<hr>

## a) Calculate Alice’s annal unconditional survival rates $_np_x$ from now to age 42. [3 marks]

In [5]:
year_range = np.arange(alice_age_anb, 44)
df1 = df.loc['${_n}p_x$', np.arange(1,len(year_range))].copy()
df1.rename(index=lambda x: year_range[x-1], inplace=True)
df1

38    0.996300
39    0.991020
40    0.983884
41    0.975226
42    0.965376
Name: ${_n}p_x$, dtype: float64

<hr>

## b) Calculate Alice’s life expectancy in years (show one decimal place)

\begin{equation}
LE = \sum_n {_n}p_x - 0.5
\end{equation}

In [6]:
LE = round((np.sum(df.loc['${_n}p_x$']) - 0.5),1)
print(f'Alice will live for another {LE} years. She will die at {round(alice_age+LE, 1)} years old')

Alice will live for another 18.9 years. She will die at 57.3 years old


<hr>

## c) Calculate the probability that Alice dies exactly between 6 and 9 years from now (show five decimal places)

Want to find $P_7$ + $P_8$ + $P_9$

In [7]:
prob = np.sum(df.loc['$P_n$', np.arange(7,10)])
print(f"Probability that Alice dies exactly between 6 and 9 years from now is {round(prob, 5)}")

Probability that Alice dies exactly between 6 and 9 years from now is 0.04176


<hr>

## d) Calculate the minimum annual premium rate (premium as a fraction of death benefit) that your company should charge for Alice’s cohort (show five decimal places).

\begin{equation}
PV = \sum^{\infty}_{n=0} \frac{_np_x\cdot\pi_n - P_n\cdot DB}{(1+r)^n}
\end{equation}
where: 
- $\pi_n$: premium to be paid at time n (up-front payment, from time 0)
- $DB$: death benefit amount
- $r$: valuation interest rate = 3.5%

In [12]:
# parameters:
# risk-free interest rate:
r = 0.015;
# deferment rate
q = 0.01;
# volatility
sigma = 0.13
# loan rate
l = 0.06;
# current house price
S = 400000;
# current loan value
K = 120000;

In [38]:
df.loc['multiplier1', 0]  = K
for t in range(1, n+1):
    df.loc['multiplier1', t] = K * np.exp((l-r)*t);

df.loc['d1', 0] = 0;
for t in range(1,n+1):
    df.loc['d1', t] = (np.log(S * np.exp(-q*t)/df.loc['multiplier1', t]) + t * sigma * sigma * 0.5)/ (sigma * np.sqrt(t))
    
for t in range(0,n+1):
    df.loc['d2', t] = df.loc['d1', t] - sigma * np.sqrt(t);

df.loc['N-d1'] = norm.cdf(-df.loc['d1'])
df.loc['N-d2'] = norm.cdf(-df.loc['d2'])

for t in range(1,n+1):
    df.loc['NNEG', t] = df.loc['$P_n$', t] * (df.loc['multiplier1', t] * df.loc['N-d2', t] - S * np.exp(-q * t) * df.loc['N-d1', t]);
    
for t in range(1,n+1):
    df.loc['$PV$', t] = df.loc['$P_n$', t] * df.loc['multiplier1', t];
    
print(f'Present value of the loan is £{round(df.loc["$PV$"].sum(), 5)}')

Present value of the loan is £161285.7075


<hr>

# e) What technologies can you apply to enable a closer prediction on the company’s cash flow linked to this line of business of permanent life insurance? (200 words)  

- smart contracts
- Internet of Things
- Big Data Analytics